In [9]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [10]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [11]:
prompt = """
Please provide a summary of the following text
Please provide your output in a manner that a 5 year old would understand

TEXT:
Philosophy (from Greek: φιλοσοφία, philosophia, 'love of wisdom') \
is the systematized study of general and fundamental questions, \
such as those about existence, reason, knowledge, values, mind, and language. \
Some sources claim the term was coined by Pythagoras (c. 570 – c. 495 BCE), \
although this theory is disputed by some. Philosophical methods include questioning, \
critical discussion, rational argument, and systematic presentation.
"""

In [12]:
num_tokens = llm.get_num_tokens(prompt)
print (f"Our prompt has {num_tokens} tokens")

Our prompt has 131 tokens


In [13]:
output = llm(prompt)
print (output)


Philosophy is when people think really hard about big questions like why we exist, how we know things, and what is important. Some people say a guy named Pythagoras made up the word a long time ago, but not everyone agrees. Philosophers use different ways to figure things out, like asking questions, talking about ideas, and using logic.
